In [ ]:
import torch
from medication_data_script import load_medication_data, LeakageOption
import random
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
WINDOW_SIZE = 400
STRIDE = 50
NUM_MED_WINDOWS_TO_SAMP = 5
data_samp_path = '/home/kuba/Documents/data/raw/kuba_watch_data/2024-07-10_06_42_46'


data = load_medication_data(WINDOW_SIZE, STRIDE, flatten=False, leakage_option=LeakageOption.FULL_LEAKAGE)

In [ ]:
test = data['test']

In [ ]:
def get_random_med_windows_from_test(windows):
    window_picks = []
    idxs_used = []

    for _ in range(NUM_MED_WINDOWS_TO_SAMP):
        pick = random.randint(0, len(test) - 1)
        while pick in idxs_used:
            pick = random.randint(0, len(test) - 1)
        
        window_picks.append(windows[pick][0]) #add window
        idxs_used.append(pick) #add so we dont reuse same windows

    return window_picks
    

In [ ]:
sampled_windows = get_random_med_windows_from_test(test)

In [ ]:
len(sampled_windows)

In [ ]:
#accl
accl_path = os.path.join(data_samp_path, "acceleration.csv")
#gyro
gyro_path = os.path.join(data_samp_path, "gyroscope.csv")

acc_with_timestamp = pd.read_csv(accl_path, skiprows=1)
acc_with_timestamp['timestamp']  = (acc_with_timestamp['timestamp'] - acc_with_timestamp['timestamp'].iloc[0]) * 1e-9 #subtract the start to get first time to be zero then convert from nano to sec
#first_row_acc = get_first_line(os.path.join(full_path, 'acceleration.csv'))

gyro_with_timestamp = pd.read_csv(gyro_path, skiprows=1)
gyro_with_timestamp['timestamp']  = (gyro_with_timestamp['timestamp'] - gyro_with_timestamp['timestamp'].iloc[0]) * 1e-9 #subtract the start to get first time to be zero then convert from nano to sec
#first_row_gyro_with_timestamp = get_first_line(os.path.join(full_path, 'gyroscope.csv'))

In [ ]:
acc_with_timestamp.iloc[:, 1:].to_numpy()

In [ ]:
#now we will ad the sameed winodws to the data 
def paste_in_windows(windows, acc, gyro):

    def is_vaild(i, idxs, full_length):
        if i < 0:
            return False
        elif (i + WINDOW_SIZE) > full_length:
            return False
        else:
            for pair in idxs:
                j, j_plus_w = pair #pais of windows we have already pasted
                if ((i + WINDOW_SIZE) >= j) and ((i + WINDOW_SIZE) <=  j_plus_w):
                    return False
                elif ((i) >= j) and ((i) <=  j_plus_w):
                    return False
            return True

    acc_augmented = acc
    gyro_augmented = gyro
    idxs = [] #list of tuples (i, i+window size) where each winodw is placed
    for window in windows:
        window_acc = window[:3,:]
        window_gyro = window[3:,:]
        print(f"Window shapes - Acc: {window_acc.T.shape}, Gyro: {window_gyro.shape}")

        valid_placment = False
        while not valid_placment:
            i = random.randint(0, len(acc) -1)
            valid_placment = is_vaild(i, idxs, len(acc))

            if valid_placment:
                acc_augmented[i:i+WINDOW_SIZE] = window_acc.T
                gyro_augmented[i:i+WINDOW_SIZE] = window_gyro.T
                idxs.append((i, i+WINDOW_SIZE)) 
    
    return acc_augmented, gyro_augmented, idxs            

In [ ]:
acc = acc_with_timestamp.iloc[:, 1:4].to_numpy()
gyro = gyro_with_timestamp.iloc[:, 1:4].to_numpy() # If we dont do this its 9points x 4) where the first index si the time stamp so we want to remove that

In [ ]:
acc = acc[:200000]
gyro = gyro[:200000]


In [ ]:
acc[0:9].T.shape

In [ ]:
acc_augmented, gyro_augmented, idxs = paste_in_windows(sampled_windows, acc, gyro)

____
lets make the windows from the augmented data

In [ ]:
def window_maker(data, window_size, stride, flatten):
    """Create windows from time series data."""
    windows = []
    
    if flatten:
        # Flatten x, y, z channels into a single vector per window
        for i in range(0, len(data[:,0]) - window_size + 1, stride):
            window = []
            window.extend(data[:,0][i:i + window_size].tolist())
            window.extend(data[:,1][i:i + window_size].tolist())
            window.extend(data[:,2][i:i + window_size].tolist())
            windows.append(window)
    else:
        # Keep x, y, z as separate channels
        for i in range(0, len(data[:,0]) - window_size + 1, stride):
            window = [
                data[:,0][i:i + window_size].tolist(),
                data[:,1][i:i + window_size].tolist(),
                data[:,2][i:i + window_size].tolist()
            ]
            windows.append(window)
            
    return windows


def process_recording(acc_data, gyro_data, window_size, stride, flatten):
    """Extract activity windows and labels from a recording."""

    if len(acc) >= window_size:
        acc_windows_batch = window_maker(acc_data, window_size, stride, flatten)
        gyro_windows_batch = window_maker(gyro_data, window_size, stride, flatten)

        # Handle potential mismatch in window counts
        min_windows = min(len(acc_windows_batch), len(gyro_windows_batch))
        acc_windows_batch = acc_windows_batch[:min_windows]
        gyro_windows_batch = gyro_windows_batch[:min_windows]
    
    return torch.tensor(acc_windows_batch), torch.tensor(gyro_windows_batch)

In [ ]:
len(acc_augmented[:,0])

In [ ]:
acc_windows, gyro_windows = process_recording(acc_augmented, gyro_augmented, window_size=WINDOW_SIZE, stride=STRIDE, flatten=False)
acc_windows.shape, gyro_windows.shape

In [ ]:
all_windows = torch.cat((acc_windows, gyro_windows), dim=1)
all_windows.shape

---- 

now lets define the model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class simple_cnn(nn.Module):
    def __init__(self, dropout=0.0):
        super().__init__()
        self.c0  = nn.Conv1d(6, 64, kernel_size=3, stride=2)

        self.c1  = nn.Conv1d(64, 128, kernel_size=3, stride=2)
        self.c2  = nn.Conv1d(128, 128, kernel_size=3)

        self.c3  = nn.Conv1d(128, 256, kernel_size=3, stride=2)
        self.c4  = nn.Conv1d(256, 256, kernel_size=3)
        self.c5  = nn.Conv1d(256, 256, kernel_size=3)

        self.gap = nn.AdaptiveAvgPool1d(1)
        self.l0 = nn.Linear(256,256)
        self.dropout = nn.Dropout(p=dropout)
        self.out = nn.Linear(256,1)
            
        
    def forward(self, x):
        x = F.relu(self.c0(x), inplace=True)
        
        x = F.relu(self.c1(x), inplace=True)
        x = F.relu(self.c2(x), inplace=True)
        
        x = F.relu(self.c3(x), inplace=True)
        x = F.relu(self.c4(x), inplace=True)
        x = F.relu(self.c5(x), inplace=True)
        
        x = self.gap(x).squeeze(-1)
        x = F.relu(self.l0(x), inplace=True)
        x = self.dropout(x)
        x = self.out(x)
        
        return x


In [ ]:
model = simple_cnn()
model.load_state_dict(torch.load('/home/kuba/projects/medication-taking-data/new_start/week00/93f1_fullleak.pth', weights_only=True))

In [ ]:
device = 'cuda:0'
model = model.to(device)
all_windows = all_windows.to(device)
with torch.no_grad():
    model.eval()
    logits = model(all_windows)

____

process the results

In [ ]:
preds = F.sigmoid(logits.cpu())
preds

In [ ]:
preds_list = preds.tolist()
preds_list = [val[0] for val in preds_list]
preds_list.reverse()

In [ ]:
counts = np.zeros((len(acc)))
sums = np.zeros((len(acc)))

len_to_use = min(len(acc) - WINDOW_SIZE, len(gyro) - WINDOW_SIZE) #Handle potential mismatch in window count
for i in range(0, len_to_use, STRIDE):
    val = preds_list.pop()
    sums[i : i+WINDOW_SIZE] += val
    counts[i : i+WINDOW_SIZE] += 1


In [ ]:
def smooth_predictions(prediction_sum, counts):
    """
    Smooth predictions by averaging, handling divisions by zero.
    """

    averaged_predictions = prediction_sum / (counts + 0.00001)
    return averaged_predictions


In [ ]:
avg_preds = smooth_predictions(sums, counts)


In [ ]:

plt.figure(figsize=(12, 6))

# Plot the accelerometer data
plt.plot(acc_augmented[:,0], label='x')
plt.plot(acc_augmented[:,1], label='y')
plt.plot(acc_augmented[:,2], label='z')
plt.plot((avg_preds * 60), label='predictions')

# Add shaded areas for each window that was pasted
for idx_pair in idxs:
    start_idx, end_idx = idx_pair
    plt.axvspan(start_idx, end_idx, color='lightgray', alpha=0.3)
    # Optionally add text labels for each window
    plt.text(start_idx, plt.ylim()[1]*0.9, f"Window", 
             fontsize=8, rotation=90, backgroundcolor='white', alpha=0.7)

plt.grid()
plt.legend()
plt.title('Accelerometer Data with Pasted Windows')
plt.xlabel('Time Points')
plt.ylabel('Acceleration')
plt.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

# Define data range
start_idx = 300000
end_idx = 450000
x_range = list(range(end_idx - start_idx))

# Add traces
fig.add_trace(go.Scatter(x=x_range, y=acc_augmented[start_idx:end_idx, 0], mode='lines', name='x'))
fig.add_trace(go.Scatter(x=x_range, y=acc_augmented[start_idx:end_idx, 1], mode='lines', name='y'))
fig.add_trace(go.Scatter(x=x_range, y=acc_augmented[start_idx:end_idx, 2], mode='lines', name='z'))
fig.add_trace(go.Scatter(x=x_range, y=(avg_preds[start_idx:end_idx] * 40), mode='lines', name='avg_preds*40'))

# Add shaded areas for each window that was pasted
for idx_pair in idxs:
    start_window, end_window = idx_pair
    
    # Only show windows that fall within our visible range
    if end_window >= start_idx and start_window <= end_idx:
        # Adjust indices to match the visible range
        visible_start = max(0, start_window - start_idx)
        visible_end = min(end_idx - start_idx, end_window - start_idx)
        
        # Calculate y-range for the shape
        y_min = min(
            np.min(acc_augmented[start_idx:end_idx, 0]),
            np.min(acc_augmented[start_idx:end_idx, 1]),
            np.min(acc_augmented[start_idx:end_idx, 2])
        ) - 0.5
        
        y_max = max(
            np.max(acc_augmented[start_idx:end_idx, 0]),
            np.max(acc_augmented[start_idx:end_idx, 1]),
            np.max(acc_augmented[start_idx:end_idx, 2])
        ) + 0.5
        
        # Add shaded rectangle
        fig.add_shape(
            type="rect",
            x0=visible_start,
            x1=visible_end,
            y0=y_min,
            y1=y_max,
            fillcolor="lightgray",
            opacity=0.3,
            layer="below",
            line_width=0,
        )
        
        # Add annotation for each window
        fig.add_annotation(
            x=visible_start + (visible_end - visible_start)/2,
            y=y_max,
            text="Window",
            showarrow=False,
            bgcolor="white",
            opacity=0.7
        )

# Update layout
fig.update_layout(
    title='Accelerometer Data with Pasted Windows',
    xaxis_title='Index',
    yaxis_title='Value',
    height=600,
    width=900
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Display the figure
fig.show()